<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/CS_GO_Analysis_RandomForest_SVM_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ls

drive/  sample_data/


In [4]:
cd 'drive/Shared drives/CS_GO_Data/processed_data'

/content/drive/Shared drives/CS_GO_Data/processed_data


In [5]:
ls

filtered_master_dmg_demos.csv       filtered_master_kills_demos.csv
filtered_master_grenades_demos.csv  filtered_meta_demos.csv


In [6]:
# Function to combine the dataframes and return a single dataframe
def combine_dataframes(df1,df2):
  df_to_return = df1.append(df2)
  df_to_return = df_to_return.reset_index(drop=True)
  return df_to_return

# Code for Preprocessing the esea_meta_demos dataset

In [7]:
def load_esea_meta_demos():
  df_esea_meta_demos = pd.read_csv("filtered_meta_demos.csv", index_col=[0])
  df_esea_meta_demos = df_esea_meta_demos.reset_index(drop=True)
  return df_esea_meta_demos

# Function to preprocess the combined esea_meta_demos dataset
def preprocess_esea_meta_demos():
  
  # load the esea_meta_demos_filtered file
  df_esea_meta_demos = load_esea_meta_demos()

  return df_esea_meta_demos

In [8]:
# function call to load and preprocess the esea_meta_demos
df_esea_meta_demos = preprocess_esea_meta_demos()

df_esea_meta_demos.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Hentai Hooligans,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Hentai Hooligans,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Hentai Hooligans,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Hentai Hooligans,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Animal Style,CounterTerrorist,ECO,5400,20550


# Code for Preprocessing the esea_master_dmg_demos dataset

In [9]:
def load_esea_master_dmg_demos():
  df_esea_master_dmg_demos = pd.read_csv("filtered_master_dmg_demos.csv")
  df_esea_master_dmg_demos.drop('Unnamed: 0', axis=1, inplace=True)
  df_esea_master_dmg_demos = df_esea_master_dmg_demos.reset_index(drop=True)
  return df_esea_master_dmg_demos

def delete_world_damage(df_esea_master_dmg_demos):
  df_esea_master_dmg_demos_cleanedup = df_esea_master_dmg_demos[df_esea_master_dmg_demos.att_side != 'None']
  df_esea_master_dmg_demos_cleanedup = df_esea_master_dmg_demos_cleanedup[df_esea_master_dmg_demos_cleanedup.att_side != df_esea_master_dmg_demos_cleanedup.vic_side]
  df_esea_master_dmg_demos_cleanedup = df_esea_master_dmg_demos_cleanedup.reset_index(drop=True)
  return df_esea_master_dmg_demos_cleanedup

# Function to preprocess the combined esea_meta_demos dataset
def preprocess_esea_master_dmg_demos():

  # load the esea_meta_demos_filtered file
  df_esea_master_dmg_demos = load_esea_master_dmg_demos()

  #delete world damage (non player damage sustained)
  df_esea_master_dmg_demos = delete_world_damage(df_esea_master_dmg_demos)

  return df_esea_master_dmg_demos

In [10]:
df_esea_master_dmg_demos = preprocess_esea_master_dmg_demos()

df_esea_master_dmg_demos.head()

,file,round,tick,seconds,att_team,vic_team,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,bomb_site,hitbox,wp,wp_type,att_id,att_rank,vic_id,vic_rank,att_pos_x,att_pos_y,vic_pos_x,vic_pos_y
0,esea_match_13770997.dem,1,15972,124.3761,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,18,9,False,NaN,Stomach,USP,Pistol,76561198048742997,0,76561198082200410,0,-1499.6900,63.33829,-669.5558,-79.769570
1,esea_match_13770997.dem,1,16058,125.0495,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,100,0,False,NaN,Head,USP,Pistol,76561198055054795,0,76561197961009213,0,-1066.8740,3.44563,-614.1868,-91.707770
2,esea_match_13770997.dem,1,16066,125.1121,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,12,7,False,NaN,RightArm,Glock,Pistol,76561198082200410,0,76561198055054795,0,-747.3146,-49.32681,-1065.5560,9.381622
3,esea_match_13770997.dem,1,16108,125.4410,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,15,7,False,NaN,Chest,USP,Pistol,76561198048742997,0,76561198082200410,0,-1501.8610,49.19798,-748.4188,-53.469220
4,esea_match_13770997.dem,1,16188,126.0674,Hentai Hooligans,Animal Style,Terrorist,CounterTerrorist,94,0,False,NaN,Head,Glock,Pistol,76561198082200410,0,76561198048742997,0,-756.2186,-80.94859,-1500.0780,60.658150


In [11]:
grouped = df_esea_master_dmg_demos.groupby('file').first()
grouped
#the first rounds of all matches

,round,tick,seconds,att_team,vic_team,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,bomb_site,hitbox,wp,wp_type,att_id,att_rank,vic_id,vic_rank,att_pos_x,att_pos_y,vic_pos_x,vic_pos_y
file,,,,,,,,,,,,,,,,,,,,,,
esea_match_13770997.dem,1,15972,124.3761,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,18,9,False,B,Stomach,USP,Pistol,76561198048742997,0,76561198082200410,0,-1499.69000,63.33829,-669.55580,-79.76957
esea_match_13779704.dem,1,20759,161.8639,Team 1,Team 2,CounterTerrorist,Terrorist,15,7,False,B,Chest,USP,Pistol,76561198078736000,0,76561198371993258,0,-170.91130,2002.93500,511.55050,1678.62100
esea_match_13779770.dem,1,22108,172.7360,Team 2,Team 1,Terrorist,CounterTerrorist,89,0,False,A,Head,Glock,Pistol,76561197988840549,0,76561197961301740,0,-1183.93800,777.64410,-2115.65300,775.35620
esea_match_13779775.dem,1,20528,159.9996,Team 2,Team 1,Terrorist,CounterTerrorist,46,15,False,A,Generic,HE,Grenade,76561198285314675,0,76561198075161204,0,257.46990,1719.66500,929.84270,1948.40800
esea_match_13779780.dem,1,20156,157.2749,Team 1,Team 2,CounterTerrorist,Terrorist,20,6,False,A,Generic,HE,Grenade,76561197961276586,0,76561198272541571,0,875.96880,2417.23200,82.23686,973.50820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
esea_match_13829155.dem,1,20872,162.9502,Team 2,Team 1,Terrorist,CounterTerrorist,100,0,False,A,Head,Glock,Pistol,76561198821543565,0,76561198434695593,0,87.94901,1821.54200,78.41760,2244.16400
esea_match_13829165.dem,1,19701,153.8882,Team 2,Team 1,Terrorist,CounterTerrorist,7,4,False,A,Stomach,Glock,Pistol,76561198007445998,0,76561198004158144,0,-465.49230,-671.70810,-431.63100,1897.77500
esea_match_13829169.dem,1,20843,162.7616,Team 2,Team 1,Terrorist,CounterTerrorist,8,4,False,B,Chest,Glock,Pistol,76561198045404066,0,76561198155685591,0,57.18853,-1427.15700,-948.27840,-2525.95100


In [12]:
rename_dict = grouped[['att_team', 'vic_team','att_side','vic_side']].copy()
rename_dict = rename_dict[((rename_dict.att_team != 'Team 1') & (rename_dict.att_team != 'Team 2')) | ((rename_dict.vic_team != 'Team 1') & (rename_dict.vic_team != 'Team 2'))]
#only consider games where the team names aren't standard. Check both columns just in case
rename_dict.reset_index(inplace=True)

In [13]:
rename_dict['Team 1'] = None
rename_dict['Team 2'] = None

In [14]:
for index,row in rename_dict.iterrows():
  if row['att_side'] == 'CounterTerrorist':
    row['Team 1'] = row['att_team']
    row['Team 2'] = row['vic_team']
  else:
    row['Team 1'] = row['vic_team']
    row['Team 2'] = row['att_team']
  
rename_dict

,file,att_team,vic_team,att_side,vic_side,Team 1,Team 2
0,esea_match_13770997.dem,Animal Style,Hentai Hooligans,CounterTerrorist,Terrorist,Animal Style,Hentai Hooligans
1,esea_match_13782025.dem,fish123,Endpoint,CounterTerrorist,Terrorist,fish123,Endpoint
2,esea_match_13782192.dem,Izako Boars,PACT,CounterTerrorist,Terrorist,Izako Boars,PACT
3,esea_match_13782504.dem,fish123,Endpoint,CounterTerrorist,Terrorist,fish123,Endpoint
4,esea_match_13782538.dem,Izako Boars,PACT,Terrorist,CounterTerrorist,PACT,Izako Boars
5,esea_match_13783103.dem,ONCA Gaming,Team Kinguin,CounterTerrorist,Terrorist,ONCA Gaming,Team Kinguin
6,esea_match_13787024.dem,Team Kinguin,PRiDE,CounterTerrorist,Terrorist,Team Kinguin,PRiDE
7,esea_match_13787180.dem,EURONICS Gaming,--to be determined--,CounterTerrorist,Terrorist,EURONICS Gaming,--to be determined--
8,esea_match_13787183.dem,ALTERNATE aTTaX,Sprout,CounterTerrorist,Terrorist,ALTERNATE aTTaX,Sprout
9,esea_match_13787381.dem,PRiDE,Team Kinguin,CounterTerrorist,Terrorist,PRiDE,Team Kinguin


In [15]:
rename_dict.drop(columns=['att_team','vic_team','att_side','vic_side'], inplace=True)
rename_dict

,file,Team 1,Team 2
0,esea_match_13770997.dem,Animal Style,Hentai Hooligans
1,esea_match_13782025.dem,fish123,Endpoint
2,esea_match_13782192.dem,Izako Boars,PACT
3,esea_match_13782504.dem,fish123,Endpoint
4,esea_match_13782538.dem,PACT,Izako Boars
5,esea_match_13783103.dem,ONCA Gaming,Team Kinguin
6,esea_match_13787024.dem,Team Kinguin,PRiDE
7,esea_match_13787180.dem,EURONICS Gaming,--to be determined--
8,esea_match_13787183.dem,ALTERNATE aTTaX,Sprout
9,esea_match_13787381.dem,PRiDE,Team Kinguin


In [16]:
df = pd.DataFrame(df_esea_master_dmg_demos.copy())


for index,dict_row in rename_dict.iterrows():
  mask = (df['file'] == dict_row['file'])
  df.loc[mask, 'att_team'] = df[mask]['att_team'].apply(lambda x: 'Team 1' if x == dict_row['Team 1'] else 'Team 2')
  df.loc[mask, 'vic_team'] = df[mask]['vic_team'].apply(lambda x: 'Team 1' if x == dict_row['Team 1'] else 'Team 2')


df_esea_master_dmg_demos_renamed = df

# Data Preprocess for Data Mining Algorithms

In [17]:
# function call to load and preprocess the esea_meta_demos
df_esea_meta_demos = preprocess_esea_meta_demos()

df_esea_meta_demos.head()

print(len(df_esea_meta_demos))

340333


In [18]:
def terrorist_counterT_win(series_shifted_winner_side):
  terrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='Terrorist' else 0)
  terrorist_win = terrorist_win.iloc[:-1]
  terrorist_win = terrorist_win.reset_index(drop=True)

  counterterrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='CounterTerrorist' else 0)
  counterterrorist_win = counterterrorist_win.iloc[:-1]
  counterterrorist_win = counterterrorist_win.reset_index(drop=True)

  return terrorist_win, counterterrorist_win

def preprocess_meta_demos_algo(df_esea_meta_demos):

  df_original_preprocessed = df_esea_meta_demos[['round','ct_eq_val','t_eq_val']]

  unique_maps = df_esea_meta_demos["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df_esea_meta_demos["map"].apply(lambda x: 1 if x == map_cs_go else 0)

  series_one_element = pd.Series([0])

  binary_values_first_round = df_esea_meta_demos["round"].apply(lambda x : 0 if x == 1 else 1)

  binary_values_first_round = binary_values_first_round.rename("not_First_Round").reset_index(drop=True)

  series_shifted_winner_side = series_one_element.append(df_esea_meta_demos["winner_side"])

  terrorist_win, counterterrorist_win = terrorist_counterT_win(series_shifted_winner_side)

  terrorist_win = (terrorist_win * binary_values_first_round).rename('terrorist_win')

  counterterrorist_win = (counterterrorist_win * binary_values_first_round).rename('counterterrorist_win')

  previous_win_df = pd.concat([terrorist_win, counterterrorist_win])

  data_to_predict_T_CT = df_esea_meta_demos['winner_side'].apply(lambda x: 1 if x=='Terrorist' else 0)

  column_names = [df_original_preprocessed, map_dataframe, terrorist_win, counterterrorist_win, data_to_predict_T_CT]

  return pd.concat(column_names, axis = 'columns', join = 'inner')

In [19]:
df_preprocessed = preprocess_meta_demos_algo(df_esea_meta_demos)

df_preprocessed

,round,ct_eq_val,t_eq_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,terrorist_win,counterterrorist_win,winner_side
0,1,4300,4250,1,0,0,0,0,0,0,0,0,0,1
1,2,6300,19400,1,0,0,0,0,0,0,0,1,0,1
2,3,7650,19250,1,0,0,0,0,0,0,0,1,0,1
3,4,24900,23400,1,0,0,0,0,0,0,0,1,0,1
4,5,5400,20550,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,4500,21950,0,0,1,0,0,0,0,0,1,0,1
340329,19,16400,26100,0,0,1,0,0,0,0,0,1,0,1
340330,20,15100,26000,0,0,1,0,0,0,0,0,1,0,1
340331,21,17550,25500,0,0,1,0,0,0,0,0,1,0,1


In [47]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [21]:
sampled = df_preprocessed.sample(n=5000, random_state=0)
X = sampled.drop('winner_side', axis=1)
y = sampled['winner_side']

In [22]:
standard_scaler = StandardScaler()
X = standard_scaler.fit_transform(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [24]:
# clf = RandomForestClassifier(random_state=0)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.678

In [26]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [41]:
parameters = {
    'bootstrap': [True, False],
    'max_depth': [1,2,3],
    'n_estimators': [10, 50, 100, 200, 500],
    'max_features': [1,2,3,4,5,6,7,8]
}

In [42]:
scores = ['precision', 'recall', 'f1']
for score in scores:
    gs = GridSearchCV(clf, parameters, scoring='%s_macro' % score)
    gs.fit(X, y)
    print(gs.best_params_)

{'bootstrap': False, 'max_depth': 1, 'max_features': 7, 'n_estimators': 100}
{'bootstrap': False, 'max_depth': 3, 'max_features': 8, 'n_estimators': 100}
{'bootstrap': False, 'max_depth': 3, 'max_features': 8, 'n_estimators': 100}


In [43]:
# clf = RandomForestClassifier(random_state=0)
clf = RandomForestClassifier(bootstrap=False, max_depth=3, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.678

In [53]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

0.645

In [54]:
neigh.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 10,
 'p': 2,
 'weights': 'uniform'}

In [56]:
parameters = {
    'weights': ['uniform','distance'],
    'algorithm': ['auto','ball_tree','kd_tree','brute'],
    'n_neighbors': [1,2,3,4,5,6,7,8, 9, 10, 11, 12]
}

In [57]:
neigh = KNeighborsClassifier()
scores = ['precision', 'recall', 'f1']
for score in scores:
    gs = GridSearchCV(neigh, parameters, scoring='%s_macro' % score)
    gs.fit(X, y)
    print(gs.best_params_)

{'algorithm': 'auto', 'n_neighbors': 8, 'weights': 'uniform'}
{'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'uniform'}
{'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'uniform'}


In [59]:
neigh = KNeighborsClassifier(n_neighbors=8)
neigh.fit(X_train, y_train)
neigh.score(X_test, y_test)

0.634

In [60]:
linear_svc = svm.SVC(kernel='linear')
linear_svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [64]:
parameters = {'C': [0.2, 0.4, 0.6, 0.8, 1.0]}

In [65]:
linear_svc = svm.SVC(kernel='linear')
scores = ['precision', 'recall', 'f1']
for score in scores:
    gs = GridSearchCV(linear_svc, parameters, scoring='%s_macro' % score)
    gs.fit(X, y)
    print(gs.best_params_)

{'C': 0.2}
{'C': 0.2}
{'C': 0.2}


In [66]:
linear_svc = svm.SVC(kernel='linear', C=0.2)
linear_svc.fit(X_train, y_train)
linear_svc.score(X_test, y_test)

0.677